# Regression test

Test the two-stage ridge regressor on a regular regression dataset.

In [80]:
from sklearn.datasets import load_boston
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge, BayesianRidge

from twostageridge import TwoStageRidge

In [81]:
data = load_boston()
X, y = data.data, data.target

In [82]:
params = {
    "twostageridge__regulariser1": [0.01, 0.1, 1., 10., 100.],
    "twostageridge__regulariser2": [0.01, 0.1, 1., 10., 100.]
}

model = make_pipeline(
    StandardScaler(),
    TwoStageRidge(treatment_col=13)
)

In [83]:
gs = GridSearchCV(model, param_grid=params, cv=5)
gs.fit(X, y)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('standardscaler', StandardScaler()),
                                       ('twostageridge',
                                        TwoStageRidge(treatment_col=13))]),
             param_grid={'twostageridge__regulariser1': [0.01, 0.1, 1.0, 10.0,
                                                         100.0],
                         'twostageridge__regulariser2': [0.01, 0.1, 1.0, 10.0,
                                                         100.0]})

In [84]:
gs.best_score_

0.4820711004593744

In [85]:
gs.best_params_

{'twostageridge__regulariser1': 100.0, 'twostageridge__regulariser2': 100.0}

In [86]:
model = make_pipeline(
    StandardScaler(),
    Ridge()
)
gs = GridSearchCV(model, param_grid={"ridge__alpha": [0.01, 0.1, 1., 10., 100.]}, cv=5)
gs.fit(X, y)
gs.best_score_

0.4820711004593853

In [87]:
gs.best_params_

{'ridge__alpha': 100.0}

In [88]:
model = make_pipeline(
    StandardScaler(),
    BayesianRidge()
)
gs = GridSearchCV(model, param_grid={}, cv=5)
gs.fit(X, y)
gs.best_score_

0.3790560205589541